In [43]:
import corruptions
import numpy
import torch
import corruption_constants
import numpy as np

In [44]:
c=("gaussian_noise","shot_noise","impulse_noise","motion_blur","zoom_blur","fog","brightness","contrast","jpeg_compression")

In [50]:
corrupt_func = c[0]
severity = 5
# print(torch.min(obs), torch.max(obs), torch.mean(obs), torch.sum(obs), torch.std(obs))
# tensor(0.) tensor(0.0299) tensor(0.0007) tensor(56.0894) tensor(0.0045)

In [51]:
def getCorruptionFunc():
    if corrupt_func in corruption_constants.CONSTANTS:
        return corruption_constants.CONSTANTS[corrupt_func]
    else:
        raise ValueError('Invalid corruption type.')

In [69]:
def applyCorruption(obs):
    Func = getCorruptionFunc()
    S = severity
    _obs = obs.numpy()

    min = np.min(_obs)
    max = np.max(_obs)
    _obs = np.interp(_obs, (min, max), (0, 255)).astype(np.uint8)


    
    _obs = Func(_obs, S)
    obs = torch.tensor(_obs).float()
    return obs

In [546]:
def argmax2d(tensor):
  '''
  Find the index of the maximum value in a 2d tensor.

  Args:
    - tensor: PyTorch tensor of size (n x 1 x d x d)

  Returns: nx2 PyTorch tensor containing indexes of max values
  '''
  n = tensor.size(0)
  d = tensor.size(2)
  m = tensor.view(n, -1).argmax(1)
  return torch.cat(((m / d).view(-1, 1), (m % d).view(-1, 1)), dim=1)

In [647]:
rnd = torch.rand(2,1,100,100)
pixel = argmax2d(rnd).long()


In [667]:
import matplotlib.pyplot as plt
import random
def translation(x, severity):
    s = torch.tensor([0.02, 0.04, 0.06, 0.08, 0.10])[severity-1]
    num_positions = (x.shape[-1] * s).int()
    assert len(x.shape) == 4
    assert x.shape[1] == 1
    for idx in range(x.shape[0]):
        heightmap = x[idx][0].clone()
        direction = random.choice(['Left', 'Right', 'Up', 'Down'])
        direction = 'Left'
        if direction == 'Left':
            translated_heightmap = torch.cat((heightmap[:, num_positions:], torch.zeros(heightmap.size(0), num_positions)), dim=1)
        if direction == 'Right':
            translated_heightmap = torch.cat((torch.zeros(heightmap.size(0), num_positions), heightmap[:, :-num_positions]), dim=1)
        if direction == 'Up':
            translated_heightmap = torch.cat((heightmap[num_positions:], torch.zeros(num_positions, heightmap.size(1))), dim=0)
        if direction == 'Down':
            translated_heightmap = torch.cat((torch.zeros(num_positions, heightmap.size(1)), heightmap[:-num_positions]), dim=0)
        x[idx][0] = translated_heightmap
        plt.figure(figsize=(8, 4))
        plt.subplot(1, 2, 1)
        plt.imshow(heightmap, cmap='gray')
        plt.title('Original Heightmap')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(translated_heightmap, cmap='gray')
        plt.title('translated Heightmap')
        plt.axis('off')

        plt.show()
    return x
rnd = torch.rand(5,1,100,100)

_ = translation(rnd, 5)


In [629]:
import random,math
import torch.nn.functional as F
def rotation(x, severity):
    angle_degrees = torch.tensor([5., 10., 15., 20., 25])[severity-1]
    if random.choice([True, False]):
        angle_degrees *= -1
    print(angle_degrees)
    assert len(x.shape) == 4
    assert x.shape[1] == 1
    for idx in range(x.shape[0]):
        heightmap = x[idx][0].clone()
        rotated_heightmap = F.affine_grid(torch.tensor([[math.cos(math.radians(angle_degrees)), math.sin(math.radians(angle_degrees)), 0],
                                                [-math.sin(math.radians(angle_degrees)), math.cos(math.radians(angle_degrees)), 0]]).unsqueeze(0),
                                   torch.Size([1, 1, heightmap.size(0), heightmap.size(1)]),
                                   align_corners=False)
        rotated_heightmap = F.grid_sample(heightmap.unsqueeze(0).unsqueeze(0), rotated_heightmap, align_corners=False)
        rotated_heightmap = rotated_heightmap.squeeze()

        x[idx][0] = rotated_heightmap
        # plt.figure(figsize=(8, 4))
        # plt.subplot(1, 2, 1)
        # plt.imshow(heightmap, cmap='gray')
        # plt.title('Original Heightmap')
        # plt.axis('off')

        # plt.subplot(1, 2, 2)
        # plt.imshow(rotated_heightmap, cmap='gray')
        # plt.title('Rotated Heightmap')
        # plt.axis('off')

        # plt.show()
    return x
_ = rotation(rnd,1)

tensor(-5.)
